In [1]:
!pip install tensorflow BeautifulSoup4 --quiet
import os
import gc
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import time
import json
import numpy as np 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pickle
import random
import matplotlib.pyplot as plt
import re
import numpy as np
from bs4 import BeautifulSoup
import unicodedata
import shutil
import plotly.express as px

fmp = '6fa0ec6e814bfee671201e723a66b46f'

def req(r):
    while True:
        got = requests.get(r).content
        try:
            got = json.loads(got.decode('utf-8'))
            time.sleep(.2)
        except: 
            continue
        if 'Error Message' in got:
            continue
        return got

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
df = pd.DataFrame(req(f'https://financialmodelingprep.com/api/v3/stock/list?apikey={fmp}'))
df = df[(df.exchangeShortName == 'NASDAQ') | (df.exchangeShortName == 'NYSE')]
df = df[df['type'] == 'stock']
df['ticker'] = df.symbol
df.head()

,symbol,name,price,exchange,exchangeShortName,type,ticker
1,CMCSA,Comcast Corporation,43.01,NASDAQ Global Select,NASDAQ,stock,CMCSA
2,KMI,"Kinder Morgan, Inc.",19.19,New York Stock Exchange,NYSE,stock,KMI
3,INTC,Intel Corporation,42.14,NASDAQ Global Select,NASDAQ,stock,INTC
4,MU,"Micron Technology, Inc.",69.75,NASDAQ Global Select,NASDAQ,stock,MU
6,GE,General Electric Company,76.24,New York Stock Exchange,NYSE,stock,GE


In [3]:
headers = {
'Cookie' : 'handl_original_ref=https%3A%2F%2Fwww.google.com%2F; handl_landing_page=https%3A%2F%2Fwww.supermonitoring.com%2Fblog%2Fcheck-browser-http-headers%2F; handl_ip=139.28.219.134; handl_ref=https%3A%2F%2Fwww.google.com%2F; handl_url=https%3A%2F%2Fwww.supermonitoring.com%2Fblog%2Fcheck-browser-http-headers%2F; _ga=GA1.2.1385115494.1641477798; _gid=GA1.2.2016070938.1641477798; _gat_gtag_UA_3825102_36=1',
'Accept-Language' : 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Accept-Encoding' : 'gzip, deflate, br',
'Referer' : 'https://www.supermonitoring.com/mini-tools/browse/',
'Sec-Fetch-Dest' : 'iframe',
'Sec-Fetch-User' : '?1',
'Sec-Fetch-Mode' : 'navigate',
'Sec-Fetch-Site' : 'same-origin',
'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
'Content-Type' : 'application/x-www-form-urlencoded',
'Origin' : 'https://www.supermonitoring.com',
'Upgrade-Insecure-Requests' : '1',
'Sec-Ch-Ua-Platform' : "Windows",
'Sec-Ch-Ua-Mobile' : '?0',
'Cache-Control' : 'max-age=0',
'Content-Length' : '483' }

##### First we need to collect all the nessesary data for the models

In [4]:
L = 10**10

def clean(txt):
    txt = re.sub(r'<table.*?/table>|<a.*?/a>|•|[_]{2,}', '', txt)
    txt = BeautifulSoup(txt, features="html.parser")
    txt = txt.get_text(strip=True, separator= '. ')
    txt = unicodedata.normalize("NFKD", txt)
    return txt

all_texts = {'ticker': [], 'quarter': [], 'calendar_year': [], 'accepted_date': [], 'text': []}

coz_fmp, fmp_fu, short = 0, None, 0
no_disman = 0
df = df.iloc[range(0, len(df))]
for idx, ticker in enumerate(df.ticker):
    sec_links = req(f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period=quarter&limit=32&apikey={fmp}")
    sec_links = sec_links[1:len(sec_links)]
    td = datetime.today()
    sec_links = [_ for _ in sec_links if (td - pd.to_datetime(_['acceptedDate']) ).days > 365 ]
    for link in sec_links:
        quarter, cal_year, acc_date = link['period'], link['calendarYear'], link['acceptedDate']
        link = link['finalLink']
        try:
            res = requests.get( link, headers=headers).text.replace('&#160;', ' ')
        except Exception as E: 
            coz_fmp += 1
            fmp_fu = ticker
            continue
        s_idx, e_idx = None, None

        man_tag = None
        dis_tag = None

        for i in re.finditer(r"<tr\b(?:(?!</tr\b).)*?Management(&#8217;|')s Discussion.*?</tr>", res, re.IGNORECASE):
            tmp = re.findall(r'href=".*?"', res[i.start():i.end()])
            if len(tmp): man_tag = tmp[0][7:-1]
            break
        for i in re.finditer(r"<tr\b(?:(?!</tr\b).)*?QUANTITATIVE AND QUALITATIVE DISCLOSURES.*?</tr>", res, re.IGNORECASE):
            tmp = re.findall(r'href=".*?"', res[i.start():i.end()])
            if len(tmp): dis_tag = tmp[0][7:-1]
            break
        links_ = []
        if man_tag is None or dis_tag is None:
            links = np.array([i for i in re.finditer(r'href=".*?"', res, re.IGNORECASE)  if res[i.start()+6] == '#' ] )
            links_ = np.array([i.end() for i in links])

            man = np.array([i.start() for i in re.finditer(r"Management(&#8217;|')s Discussion", res, re.IGNORECASE)])
            
            dis = np.array([i.start() for i in re.finditer(r'QUANTITATIVE AND QUALITATIVE DISCLOSURES', res, re.IGNORECASE)])

            if len(dis) == 0 or len(man) == 0: 
                no_disman += 1
                continue


        if len(links_) != 0 or not (man_tag is None or dis_tag is None):
            if man_tag is None or dis_tag is None:
                mind = np.inf
                for i in man:
                    diff = i - links_
                    diff[diff < 0] = L
                    mi = np.argmin(diff)
                    if diff[mi] < mind:
                        mind = diff[mi]
                        man_tag = res[links[mi].start()+7:links[mi].end()-1]

                mind = np.inf
                for i in dis:
                    diff = i - links_
                    diff[diff < 0] = L
                    mi = np.argmin(diff)
                    if diff[mi] < mind:
                        mind = diff[mi]
                        dis_tag = res[links[mi].start()+7:links[mi].end()-1]

            if not (man_tag is None or dis_tag is None):
                for i in re.finditer(f'(id|name)="{man_tag}"', res, re.IGNORECASE):    
                    s_idx = i.end()
                    break
                for i in re.finditer(f'(id|name)="{dis_tag}"', res, re.IGNORECASE):    
                    e_idx = i.start()
                    break

                if not (s_idx is None or e_idx is None  or s_idx > e_idx):
                    if e_idx - s_idx < 1000: 
                        short += 1
                
                    all_texts['ticker'].append(ticker)
                    all_texts['quarter'].append(quarter)
                    all_texts['calendar_year'].append(cal_year)
                    all_texts['accepted_date'].append(acc_date)
                    all_texts['text'].append(clean(res[s_idx:e_idx]))

                    continue
            
        
        broke = False
        for i in man:
            for j in dis:
                if i > j: continue
                txt = BeautifulSoup(res[i:j], features="html.parser")
                if len(txt.get_text()) < 200: continue

                all_texts['ticker'].append(ticker)
                all_texts['quarter'].append(quarter)
                all_texts['calendar_year'].append(cal_year)
                all_texts['accepted_date'].append(acc_date)
                all_texts['text'].append(clean(res[i:j]))


                total, used, free = shutil.disk_usage("/work")
                assert free >= .1, 'No space'

                broke = True
                break
            if broke: break

df = pd.DataFrame(all_texts)

In [5]:
df.text = df.text.apply(lambda x: re.sub(r'20(1[0-9]|2[0-2])', '', x))

# Backtesting & Models

In [6]:
df['time'] = pd.to_datetime(df.accepted_date)
df = df.drop(columns = ['accepted_date'])
df = df.dropna()
df['length'] = df.text.apply(lambda x: x.count(' ')+1)
df = df[df.length < 25000]
df = df[df.time < datetime.today() - timedelta(days = 366)]
df = df[df.calendar_year.astype(int) > 2015]
df['y'] = 0
df['y'][df.change > 1.] = 1
df = df[df.change < 5]

In [7]:
oov_tok = "<OOV>"
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(df.text)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 10000
trunc_type='post'
padding_type='post'

def backtest(year, tokenizer):
    train = df[(df.time < pd.to_datetime(f'01-01-{year}'))]
    valid = df[(df.time < pd.to_datetime(f'01-01-{year+1}')) & (df.time >= pd.to_datetime(f'01-01-{year}'))]

    vocab_size = len(tokenizer.word_index) + 1
    embedding_dim = 8
    max_length = 10000
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"

    training_sequences = tokenizer.texts_to_sequences(train.text)
    training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    testing_sequences = tokenizer.texts_to_sequences(valid.text)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    random.seed(1)

    tar = 1.
    vocab_size = len(tokenizer.word_index) + 1

    train['y'] = 0
    train['y'][train.change >= tar] = 1

    valid['y'] = 0
    valid['y'][valid.change >= tar] = 1

    y_train = np.array(train.y)
    y_test = np.array(valid.y)
    embedding_dim = 8
    max_length = 10000
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"

    num_epochs = 3
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(12, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC'])
    model.fit(training_padded, y_train, epochs=num_epochs, verbose=0) #validation_data=(testing_padded, y_test),
    predicted_proba_hg = model.predict(testing_padded)
    valid['proba'] = predicted_proba_hg
    chng = valid.sort_values(by = 'proba', ascending=False).iloc[range(0, int(0.2*len(valid)))].change.mean()
    sp5 = pd.DataFrame(req(f'https://financialmodelingprep.com/api/v3/historical-price-full/^GSPC?from={pd.to_datetime(f"01-01-{year}")}&to={pd.to_datetime(f"01-01-{year+1}")}&apikey={fmp}')['historical'])
    sp5['date'] = pd.to_datetime(sp5.date)
    chngsp = sp5.close.iloc[0]/sp5.close.iloc[-1]
    return chng - chngsp, model

In [8]:
gc.collect()

1667

In [9]:
models = dict()
avg_lst = []
for year in range(2016, 2022):
    back, models[year] = backtest(year, tokenizer)
    print(year, round(back,3))
    avg_lst.append(round(back,3))
    gc.collect()
print(sum(avg_lst)/len(avg_lst))

2016 0.104
2017 0.133
2018 0.089
2019 0.136
2020 0.148
2021 0.357
0.16116666666666665


In [10]:
gc.collect()

1390

##### Performing backtesting since it ensures that the models are not overfitted

In [11]:
def backtest_negativity(year, tokenizer):
    train = df[(df.time < pd.to_datetime(f'01-01-{year}'))]
    valid = df[(df.time < pd.to_datetime(f'01-01-{year+1}')) & (df.time >= pd.to_datetime(f'01-01-{year}'))]

    vocab_size = len(tokenizer.word_index) + 1
    embedding_dim = 8
    max_length = 10000
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"

    training_sequences = tokenizer.texts_to_sequences(train.text)
    training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    testing_sequences = tokenizer.texts_to_sequences(valid.text)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    random.seed(1)

    tar = .9
    vocab_size = len(tokenizer.word_index) + 1

    train['y'] = 0
    train['y'][train.change <= tar] = 1

    valid['y'] = 0
    valid['y'][valid.change <= tar] = 1

    y_train = np.array(train.y)
    y_test = np.array(valid.y)
    embedding_dim = 8
    max_length = 10000
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"

    num_epochs = 3
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Conv1D(32, 2),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(125, activation='relu'),
        tf.keras.layers.Dense(40, activation='relu'),
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(15, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC'])
    model.fit(training_padded, y_train, epochs=num_epochs, verbose=0) #validation_data=(testing_padded, y_test),
    predicted_proba_hg = model.predict(testing_padded)
    valid['proba'] = predicted_proba_hg
    chng = valid.sort_values(by = 'proba', ascending=False).iloc[range(0, int(0.2*len(valid)))].change.mean()
    return chng, model

In [12]:
gc.collect()

12620

In [13]:
models2 = dict()
avg_lst = []
for year in range(2016, 2022):
    back, models2[year] = backtest_negativity(year, tokenizer)
    print(year, round(back, 3))
    avg_lst.append(round(back, 3))
    gc.collect()
print(sum(avg_lst)/len(avg_lst))

2016 0.891
2017 0.957
2018 0.884
2019 0.892
2020 0.978
2021 0.983
0.9308333333333332


# Visualisation

In [14]:
years = list(range(2016, 2022))
avg, nf = [], []
for year in years:
    sub_df = df[df.time < pd.to_datetime(f'31-12-{year}')]
    testing_sequences = tokenizer.texts_to_sequences(sub_df.text)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    sub_df['preds'] = models[year].predict(testing_padded)[:,0]
    perc = sub_df.sort_values(by='preds', ascending= False).iloc[int(0.2*len(sub_df))].preds
    avg.append(np.sum((sub_df.preds >= perc) & (sub_df.change > 1)) / np.sum(sub_df.preds >= perc))
    nf.append(np.sum(sub_df.change > 1)/len(sub_df))


##### Let us visualize the percentage of stocks that grew after one year for market and for ones which were deemed to be positive

In [15]:
bar = pd.DataFrame({'info': years, 'Stocks with high positivity': avg, 'Market': nf})

fig = px.bar(bar, x='info', y=['Market', 'Stocks with high positivity'], barmode="group", labels={'value':'Percentage'}, hover_data={'info':False, # remove species from hover data
                             'variable':False})
fig.update_layout(yaxis= { 'tickformat': ',.0%'})
fig.update_layout( autosize=False, width=580, height=580)
fig.update_xaxes(title = '')
fig.update_layout(legend_title_text='')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(template='plotly_white')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor='auto',
    x=0.2
))
fig.show()

In [16]:
dic = {'Year':[], 'Small':[], 'Medium':[], 'Large':[], 'Severe':[]}
for i in years:
    yr = df[df.time < pd.to_datetime(f'31-12-{i}')]
    testing_sequences = tokenizer.texts_to_sequences(yr.text)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    yr['negativity'] = models2[year].predict(testing_padded)[:,0]
    new = yr.dropna().sort_values(by = 'negativity').iloc[range(round(0.2*len(yr)))]
    new = new[new.change < 1]
    dic['Year'].append(i)
    dic['Small'].append(len(new[new.change > 0.9])/len(new))
    dic['Medium'].append(len(new[(new.change > 0.75) & (new.change <= 0.9)])/len(new))
    dic['Large'].append(len(new[(new.change > 0.5) & (new.change <= 0.75)])/len(new))
    dic['Severe'].append(len(new[(new.change <= 0.5)])/len(new))

##### Plotting the distribution to see how effective the predictions are

In [17]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.express as px
df = pd.DataFrame(dic)
fig = make_subplots(rows=2, cols=3, specs=[[{"type": "pie"}]*3, [{"type": "pie"}]*3])
names = ['Small', 'Medium', 'Large', 'Severe']
j, k = 1, 0
for i in df.Year:
    if k == 3:
        j+=1
        k = 0
    k+=1

    fig.add_trace(   go.Pie(labels=names, values=list(df[df.Year == i][names].values[0]), title = i ), row=j, col = k)
    fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(title = 'Decline distributions of stocks with low negativity')
fig.show()

In [18]:
df = pd.read_pickle('blyat_kak_xochesh.pkl')

In [19]:
dic = {'Year':[], 'Small':[], 'Medium':[], 'Large':[], 'Severe':[]}
for i in years:
    new = df[df.time < pd.to_datetime(f'31-12-{i}')].dropna()
    new = new[new.change < 1]
    dic['Year'].append(i)
    dic['Small'].append(len(new[new.change > 0.9])/len(new))
    dic['Medium'].append(len(new[(new.change > 0.75) & (new.change <= 0.9)])/len(new))
    dic['Large'].append(len(new[(new.change > 0.5) & (new.change <= 0.75)])/len(new))
    dic['Severe'].append(len(new[(new.change <= 0.5)])/len(new))

##### Comparing to the market

In [20]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.express as px
df = pd.DataFrame(dic)
fig = make_subplots(rows=2, cols=3, specs=[[{"type": "pie"}]*3, [{"type": "pie"}]*3])
names = ['Small', 'Medium', 'Large', 'Severe']
j, k = 1, 0
for i in df.Year:
    if k == 3:
        j+=1
        k = 0
    k+=1

    fig.add_trace(   go.Pie(labels=names, values=list(df[df.Year == i][names].values[0]), title = i ), row=j, col = k)
    fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(title = 'Decline distributions of the market')
fig.show()

# Adding financials

##### Collecting the financial data

In [21]:

dic = {'ticker' : [], 'quarter' : [], 'year' : [], 'time' : []}
cash1 = req(f"https://financialmodelingprep.com/api/v3/cash-flow-statement/AAPL?period=quarter&limit=1000000000&apikey={fmp}")[0]
balance1 = req(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/AAPL?period=quarter&limit=1000000000&apikey={fmp}")[0]
incm1 = req(f"https://financialmodelingprep.com/api/v3/income-statement/AAPL?period=quarter&limit=1000000000&apikey={fmp}")[0]
odin, dva, thor = True, True, True
for k in cash1:
    if k in ['date', 'symbol', 'reportedCurrency', 'fillingDate', 'acceptedDate', 'period', 'link', 'finalLink']: continue
    try: 
        if odin:
            dic[k] = []
    except: 
        dic[k].append(np.nan)
odin = False

for k in balance1:
    if k in ['date', 'symbol', 'reportedCurrency', 'fillingDate', 'acceptedDate', 'period', 'link', 'finalLink']: continue
    if k not in cash1.keys():
        try : 
            if dva:
                dic[k] = []
        except: 
            dic[k].append(np.nan)
dva = False


for k in incm1:
    if k in ['date', 'symbol', 'reportedCurrency', 'fillingDate', 'acceptedDate', 'period', 'link', 'finalLink']: continue
    if (k not in cash1.keys()) & (k not in balance1.keys()):
        try : 
            if thor:
                dic[k] = []
        except: 
            dic[k].append(np.nan)
thor = False

df['date'] = df.time.apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
cache_cash, cache_peers, cache_shrs, cache_balance, cache_incm = {}, {}, {}, {}, {}
PFCF_discount, EVE_discount, PS_discount, PE_discount = [], [], [], []
odin, dva, thor = False, False, False

for i in range(0, len(df)):
    if df.ticker.iloc[i] in cache_cash: cash = cache_cash[df.ticker.iloc[i]]
    else: 
        cash = req(f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{df.ticker.iloc[i]}?period=quarter&limit=1000000000&apikey={fmp}")
        cache_cash[df.ticker.iloc[i]] = cash
    for j in cash:
        if (j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.calendar_year.iloc[i]):
            break
    if not ((j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.calendar_year.iloc[i])):
        continue
    cash1 = j
    for j in cash:
        if (j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(cash1['calendarYear'])-1)):
            break
    if not ((j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(cash1['calendarYear'])-1))):
        continue
    cash2 = j
    if type(pd.to_datetime(cash1['acceptedDate'])) == type(None): continue
    if (df.time.iloc[i] - pd.to_datetime(cash1['acceptedDate']) < timedelta(days = -2)) & (df.time.iloc[i] - pd.to_datetime(cash1['acceptedDate']) > timedelta(days = 30)): continue

    
    if df.ticker.iloc[i] in cache_shrs: shrs = cache_shrs[df.ticker.iloc[i]]
    else: 
        zapr = req(f"https://financialmodelingprep.com/api/v3/quote/{df.ticker.iloc[i]}?apikey={fmp}")
        if len(zapr) == 0: continue
        shrs = zapr[0]['sharesOutstanding']
        cache_shrs[df.ticker.iloc[i]] = shrs
    
    prc = req(f'https://financialmodelingprep.com/api/v3/historical-price-full/{df.ticker.iloc[i]}?from={df.time.iloc[i] - timedelta(days = 5)}&to={df.time.iloc[i]}&apikey={fmp}')
    if len(prc) == 0:
        continue
    prc = prc['historical'][0]['close']
    if (type(shrs) == type(None)) or (type(prc) == type(None)): continue
    mcap = prc*shrs


    
    if df.ticker.iloc[i] in cache_balance: balance = cache_balance[df.ticker.iloc[i]]
    else:
        balance = req(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{df.ticker.iloc[i]}?period=quarter&limit=1000000000&apikey={fmp}")
        cache_balance[df.ticker.iloc[i]] = balance

    for j in balance:
        if (j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.calendar_year.iloc[i]):
            break
    if not ((j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.calendar_year.iloc[i])):
        continue
    balance1 = j
    for j in balance:
        # if j['date'] == '1633-08-19': continue
        if (j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(balance1['calendarYear'])-1)):
            break
    if not ((j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(balance1['calendarYear'])-1))):
            continue
    balance2 = j


    if df.ticker.iloc[i] in cache_incm: incm = cache_incm[df.ticker.iloc[i]]
    else:
        incm = req(f"https://financialmodelingprep.com/api/v3/income-statement/{df.ticker.iloc[i]}?period=quarter&limit=1000000000&apikey={fmp}")

    for j in incm:
        if (j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.calendar_year.iloc[i]):
            break
    if not ((j['period'] == 'Q' + df.quarter.iloc[i]) & (j['calendarYear'] == df.calendar_year.iloc[i])):
        continue
    incm1 = j

    for j in incm:
        if (j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(incm1['calendarYear'])-1)):
            break
    if not ((j['period'] == 'Q' +str(df.quarter.iloc[i])) and (j['calendarYear'] == str(int(incm1['calendarYear'])-1))):
        continue
    incm2 = j


    for k in dic:
        if (k in cash1.keys()):
            try: 
                if cash2[k] != 0:
                    if (float(cash1[k]) > 0) and (float(cash2[k]) > 0):
                        dic[k].append(float(cash1[k])/float(cash2[k]))
                    elif (float(cash1[k]) < 0) or (float(cash2[k]) < 0):
                        dic[k].append(-999)
                    else:
                        dic[k].append(1-(abs(float(cash1[k]))/abs(float(cash2[k]))))
                else:
                    if cash1[k] == 0:
                        dic[k].append(0)
                    else:
                        dic[k].append(-1)
            except: 
                dic[k].append(np.nan)
    for k in dic:
        if (k in balance1.keys()) and (k not in cash1.keys()):
            try: 
                if balance2[k] != 0:
                    if (float(balance1[k]) > 0) & (float(balance2[k]) > 0):
                        dic[k].append(float(balance1[k])/float(balance2[k]))
                    elif (float(balance1[k]) < 0) & (float(balance2[k]) > 0) or (float(balance1[k]) > 0) & (float(balance2[k]) < 0):
                        dic[k].append(-999)
                    else:
                        dic[k].append(1-(abs(float(balance1[k]))/abs(float(balance2[k]))))
                else:
                    if balance1[k] == 0:
                        dic[k].append(0)
                    else:
                        dic[k].append(-1)
            except: 
                dic[k].append(np.nan)

    for k in dic:
        if (k in incm1.keys() and (k not in cash1.keys())) and (k not in balance1.keys()):
            try: 
                if (k not in incm1.keys()) and (k not in cash1.keys()) and (k not in balance1.keys()) and (k not in ['ticker', 'year', 'quarter', 'time']):
                    dic[k].append(np.nan) 
                if incm2[k] != 0:
                    if (float(incm1[k]) > 0) & (float(incm2[k]) > 0):
                        dic[k].append(float(incm1[k])/float(incm2[k]))
                    elif (float(incm1[k]) < 0) & (float(incm2[k]) > 0) or (float(incm1[k]) > 0) & (float(incm2[k]) < 0):
                        dic[k].append(-999)
                    else:
                        dic[k].append(1-(abs(float(incm1[k])))/abs(float(incm2[k])))
                else:
                    if incm1[k] == 0:
                        dic[k].append(0)
                    else:
                        dic[k].append(-1)
            except: 
                dic[k].append(np.nan)

    dic['ticker'].append(df.ticker.iloc[i])
    dic['quarter'].append(df.quarter.iloc[i])
    dic['year'].append(df.calendar_year.iloc[i])
    dic['time'].append(df.time.iloc[i])
new = pd.DataFrame(dic)

##### Merging the collected data with existing predictions to combine the predictive power

In [22]:
new['quarter'] = 'Q' + new.quarter
df = pd.read_pickle('sec_texts.pkl')
df = new.merge(df, left_on = ['ticker', 'quarter', 'year'], right_on = ['ticker', 'quarter', 'calendar_year'])
df = df.drop(columns=['time_y'])
df.rename(columns={'time_x':'time'}, inplace=True)
df['positivity'] = 0

In [23]:
for year in years:
    yr = df[df.time < pd.to_datetime(f'31-12-{i}')]
    testing_sequences = tokenizer.texts_to_sequences(yr.text)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    df['positivity'][df.time < pd.to_datetime(f'31-12-{i}')] = models[year].predict(testing_padded)[:,0]

In [24]:
df['y'] = 0
df['y'][df.change >= 1] = 1

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='y'), df.y, test_size=0.33, random_state=0)
check = X_test.copy()
X_train, X_test = X_train.drop(columns = ['ticker', 'year', 'quarter', 'time', 'change', 'cik','calendarYear', 'accepted_date', 'text']), X_test.drop(columns = ['ticker', 'year', 'quarter', 'time', 'change', 'cik', 'calendarYear', 'accepted_date', 'text'])
rf = RandomForestClassifier(random_state=1).fit(X_train, y_train)
check['probability'] = rf.predict_proba(X_test)[:,1]

##### Checking the resulted accuracy

In [26]:
check['y'] = y_test
avg_lst = []
for i in check.year.unique():
    dat = round(np.mean(round(check[check.year == i].probability) == check[check.year == i].y), 3)
    avg_lst.append(dat)
    print(i, dat)
print(sum(avg_lst)/len(avg_lst))

2016 0.745
2017 0.763
2018 0.713
2019 0.827
2020 0.804
2021 0.768
0.77


In [27]:
for year in years:
    yr = df[df.time < pd.to_datetime(f'31-12-{i}')]
    testing_sequences = tokenizer.texts_to_sequences(yr.text)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    df['negativity'][df.time < pd.to_datetime(f'31-12-{i}')] = models2[year].predict(testing_padded)[:,0]

In [28]:
df['negativity'] = 0
df['y'] = 0
df['y'][df.change <= 0.9] = 1
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='y'), df.y, test_size=0.33, random_state=0)
check = X_test.copy()
X_train, X_test = X_train.drop(columns = ['ticker', 'year', 'quarter', 'time', 'change', 'cik','calendarYear', 'accepted_date', 'text']), X_test.drop(columns = ['ticker', 'year', 'quarter', 'time', 'change', 'cik', 'calendarYear', 'accepted_date', 'text'])
rf = RandomForestClassifier(random_state=1).fit(X_train, y_train)
check['probability_neg'] = rf.predict_proba(X_test)[:,1]

In [29]:
check['y'] = y_test
avg_lst = []
for i in check.year.unique():
    dat = round(np.mean(round(check[check.year == i].probability_neg) == check[check.year == i].y), 3)
    avg_lst(dat)
    print(i, dat)
print(sum(avg_lst)/len(avg_lst))

2016 0.733
2017 0.772
2018 0.735
2019 0.834
2020 0.851
2021 0.835
0.7933333333333333
